<a href="https://colab.research.google.com/github/babypotatotang/Introduction-to-DeepLearning/blob/main/7.%20%ED%95%A9%EC%84%B1%EA%B3%B1%20%EC%8B%A0%EA%B2%BD%EB%A7%9D(Convolutional%20Neural%20Network)/02_CNN%EC%9C%BC%EB%A1%9C_MNIST_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 모델 이해하기 
--- 
## (1) 첫번째 표기방법
* 합성곱(`nn.Conv2d`) + 활성화 함수(`nn.ReLU`)를 하나의 합성곱 층으로 보고 맥스 풀링(`nn.MaxPool2d`)은 풀링층으로 별도로 부름

## (2) 두번째 표기방법
* 합성곱 + 활성화 함수 + 맥스 풀링을 하나의 합성곱 층으로 봄. 

맥스 풀링을 하나의 합성곱층으로 보느냐 아니냐의 차이로, 이번에는 두번째 표기방법을 사용하여 모델 정의 

## 모델 아키텍쳐
```python
# 1번 레이어: 합성곱층 
합성곱(in_channerl = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1 ) + relu 맥스풀링 (kernel_size = 2, stride = 2) 

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax
```

# 2. 모델 구현하기 
--- 

In [ ]:
import torch
import torch.nn as nn 

In [ ]:
inputs = torch.Tensor(1,1,28,28) # 배치 크기 x 채널 x 높이 x 너비 크기의 텐서 선언
print(f'텐서의 크기 : {inputs.shape}')

### 합성곱층과 풀링 선언하기

In [ ]:
conv1 = nn.Conv2d(1,32,3,padding = 1)
print(conv1)

conv2 = nn.Conv2d(32,64,kernel_size = 3, padding = 1)
print(conv2)

pool = nn.MaxPool2d(2)
print(pool)

### 구현체를 연결하여 모델 만들기 

In [ ]:
out = conv1(inputs)
out = pool(out)
out = conv2(out)
out = pool(out)
print(out.shape)

In [ ]:
out = out.view(out.size(0), -1) # 첫번째 차원인 배치 차원은 그대로 두고 나머지를 일렬로 펼치기 
print(out.shape)

In [ ]:
fc = nn.Linear(3136,10)
out = fc(out)
print(out.shape)

# 3. CNN으로 MNIST 분류하기 

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
lr = 0.001
epochs = 15
batch_size = 100

In [ ]:
from torch._C import ThroughputBenchmark
mnist_train = dsets.MNIST(root = 'MNIST_data/', 
                          train = True, transform = transforms.ToTensor(), download = True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size, 
                                          shuffle = True, 
                                          drop_last = True)

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [ ]:
model = CNN().to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

In [ ]:
for epoch in range(epochs):
    avg_cost = 0

    for X, Y in data_loader: 
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())